# Języki i Biblioteki Analizy Danych

## Laboratorium 2.: Wyjątki

#### mgr inż. Zbigniew Kaleta

Wyjątki służą do __sygnalizacji__ sytuacji błędnej, awaryjnej, _wyjątkowej_.

Jeśli w programie zdarzy się coś, co się zdarzyć nie powinno, to najprawdopodobniej nie da się kontynuować normalnego działania, bo będzie ono dawało błędne wyniki. Wtedy __rzucamy__ wyjątek.

Rzucenie wyjątku przerywa normalne wykonanie programu.

Przykłady sytuacji awaryjnych:
- próba dzielenia przez 0
- wyjście poza zakres tablicy
- próba otwarcia nieistniejącego pliku
- w miejscu gdzie spodziewaliśmy się liczby znaleźliśmy napis
- brak miejsca na dysku

W niektórych sytuacjach Pythona sam sygnalizuje błąd wykonania przez rzucenie wyjątku.

In [1]:
5 + '5'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

W innych robimy to my

In [2]:
radius = -1

if radius <= 0:
    raise ValueError("Radius must be a positive number")

ValueError: Radius must be a positive number

In [4]:
x = 2

if x < 4:
    raise ValueError('Low variable') # The raise statement allows the programmer to force a specified exception to occur.

ValueError: Low variable

In [5]:
raise ValueError('Hello there!')

ValueError: Hello there!

Zdecydowana większość typów wyjątków w Pythonie ma nazwy kończące się słowem Error, a nie Exception. Nie istnieje rozróżnienie na błędy i wyjątki jak w Javie.

Jeśli wyjątek nie zostanie __złapany__, to zakończy działanie programu z komunikatem o błędzie.

Ponieważ taki komunikat jest mało czytelny dla użytkownika końcowego, zwykle chcemy wyjątek złapać i __obsłużyć__, nawet jeśli jedyne co zrobimy to wyświetlenie bardziej czytelnego komunikatu i zakończenie programu.

Do łapania wyjątków w Pythonie służy konstrukcja try-except

In [6]:
try:
    5 + '5'
except TypeError:
    print('no-no')

no-no


Jeżeli w bloku try (bezpośrednio lub we wnętrzu wywołanej z niego funkcji) zostanie rzucony wyjątek, jego wykonanie natychmiast jest przerywane i jest wykonywany blok except, pasujący do typu rzuconego wyjątku.

In [7]:
def foo():
    5 + '5'
    print("Hello")
    
try:
    foo()
    print("world!")
except TypeError:
    print('no-no')
print("The end!")

no-no
The end!


Proszę zauważyć, że żaden z komunikatów "Hello" ani "world!" nie został wypisany - wykonanie kodu w bloku try zostało przerwane i nie zostało wznowione.

Czasami obiekt wyjątku zawiera jakieś dodatkowe informacje, po które chcemy sięgnąć, więc potrzebujemy dostępu do niego.

In [8]:
try:
    5 + '5'
except TypeError as e:
    print(type(e), e)

<class 'TypeError'> unsupported operand type(s) for +: 'int' and 'str'


W Pythonie 2 to samo można było osiągąć przy pomocy innej składni:

In [ ]:
%%python2

try:
    [][0]
except IndexError, e:
    print(e)

Jeden blok try może mieć wiele bloków except, obsługujących wyjątki różnych typów.

In [9]:
try:
    dict()[3]
except TypeError as e:
    print(type(e), e)
except KeyError as e:
    print("it's only key error")

it's only key error


Jeżeli chcemy wyjątki kilku typów obsłużyć w ten sam sposób, możemy to zrobić przy pomocy jednego bloku except, wymieniając wszystkie obsługiwane typy w postaci krotki. W tym przypadku nawiasy wokół krotki są konieczne.

In [10]:
try:
    5 + '5'
except (TypeError, KeyError) as e:
    print(type(e), e)

<class 'TypeError'> unsupported operand type(s) for +: 'int' and 'str'


In [ ]:
try:
    5 + '5'
except TypeError, KeyError as e:
    print(type(e), e)

Można też deklarować własne typy wyjątków.

In [12]:
class WrongAnswerError(Exception): # zawsze musi być dziedziczenie klasy Exception dla nowych wyjątków
    pass

try:
    raise WrongAnswerError('wrong wrong wrong!') # podnieś błąd 
except WrongAnswerError: # błąd podniesiony i wykonuje except
    print('good good good')

good good good


Korzyści z własnych typów wyjątków:
- możemy mu nadać nazwę oddającą charakter błędu (zamiast stosować jokera w postaci `ValueError`)
- pozwalamy sobie na sygnalizowanie różnych błędów i obsługiwanie ich osobnymi blokami `except`

Każdy wyjątek __musi__ dziedziczyć z klasy `BaseException`, a zwykle dziedziczy z `Exception` (`Exception` jest bezpośrednim potomkiem `BaseException`, więc wymóg dziedziczenia z `BaseException` jest tutaj spełniony).

In [15]:
class A(Exception):
    pass

raise A('a')

A: a

exception Exception

    All built-in, non-system-exiting exceptions are derived from this class. All user-defined exceptions should also be derived from this class.

Blok `try` może także posiadać bloki `else` i `finally`.

Blok `else` jest wykonywany bezpośrednio po bloku `try`, jeżeli wykonanie dojdzie normalnie do jego końca (czyli nie zostanie rzucony wyjątek, ani wykonany `return`).

Blok `finally` jest wykonywany zawsze:
- jeżeli w bloku `try` nie wystąpił wyjątek, to bezpośrednio po `try`, lub `else`
- jeżeli wystąpił i został obsłużony w bloku `except`, to bezpośrednio po tym bloku `except`
- jeżeli wystąpił i nie został obsłużony, to bezpośrednio po bloku `try`

Blok `else` jest rzadko wykorzystywany - zwykle można osiągnąć ten sam skutek umieszczając kod po całej konstrukcji try-except.

Po bloku `try` może być dowolnie dużo bloków `except` (każdy obsługujący inny typ wyjątku), jeden blok `else` i jeden blok `finally` (w tej kolejności), przy czym __musi__ wystąpić przynajmniej jeden `except` lub `finally`.

In [ ]:
try:
    pass
except WrongAnswerError:
    print('good good good')
else:
    print('Uhm...')
finally:
    print('bye!')

Uwaga na funkcje i instrukcję `return`: `return` powinien być albo w bloku `try`, albo `except`, albo `finally`, ale nigdy więcej niż w jednym z nich. Użycie kilku `return`'ów w różnych blokach jest formalnie poprawne, ale to piekło do debugowania.

Analogicznie, należy unikać rzucania wyjątku w bloku `finally` czy `else`, choć to jest mniej problematyczne.

In [ ]:
try:
    pass
except WrongAnswerError:
    print('good good good')
else:
    raise WrongAnswerError('wrong wrong wrong!')
finally:
    print('bye!')

Typy wyjątków, które warto znać:
- TypeError - Raised when an operation or function is applied to an object of inappropriate type. The associated value is a string giving details about the type mismatch.
- ValueError - Raised when an operation or function receives an argument that has the right type but an inappropriate value, and the situation is not described by a more precise exception such as IndexError.
- IndexError - Raised when a sequence subscript is out of range.
- KeyError - Raised when a mapping (dictionary) key is not found in the set of existing keys.
- NameError - Raised when a local or global name is not found. This applies only to unqualified names. The associated value is an error message that includes the name that could not be found
- AttributeError - Raised when an attribute reference or assignment fails.
- ImportError - Raised when the import statement has troubles trying to load a module. Also raised when the “from list” in from ... import has a name that cannot be found.

[https://docs.python.org/3/library/exceptions.html#exception-hierarchy]

## Jak wyglądał świat przed wynalezieniem wyjątków, czyli obsługa socketów w C

Źródło: [https://www.thegeekstuff.com/2011/12/c-socket-programming/?utm_source=feedburner]

Co robi powyższy kod? Obsługuje wyjątki w C 

## Asercje

Asercje są łatwym i wygodnym sposobem dodania kontroli poprawności działania do naszego kodu.
Przyjmują postać: `assert <warunek, który oczekujemy, że będzie prawdziwy>, <komunikat dlaczego ma być prawdziwy>`

Przykładowo zamiast przedstawionego wcześniej kodu:

In [16]:
radius = -1

if radius <= 0:
    raise ValueError("Radius must be a positive number")

ValueError: Radius must be a positive number

Możemy napisać krócej:

In [17]:
radius = -1

assert radius > 0, "Radius must be a positive number"

AssertionError: Radius must be a positive number

Najważniejsza różnica jest taka, że nie mamy możliwości wyboru typu wyjątku, jaki zostanie rzucony - zawsze będzie to `AssertionError`.

Asercje służą przede wszystkim do sprawdzania poprawności argumentów wywołania funkcji, lub wyników pośrednich (jeżeli w kodzie jest błąd, który powoduje błędne wyniki, to używanie asercji pozwoli nam szybciej go wykryć i zawęzić obszar poszukiwań).

Technicznie `AssertionError` jest normalnym wyjątkiem.

In [18]:
AssertionError.__bases__

(Exception,)

In [19]:
try:
    assert 0 > 1
except:
    pass

## Zasady postępowania z wyjątkami

#### Blok except musi mieć możliwie specyficzny wyjątek

Proszę __nigdy__ nie używać konstrukcji `except Exception:`, lub (jeszcze gorzej) `except:` (bez podanego typu wyjątku). 

Powoduje to wrzucenie wszystkich możliwych błędów do jednego worka, włącznie z `AssertionError`, `NameError` czy `KeyboardInterrupt` i ogromnie obniża czytelność kodu.

A za rzucenie `Exception` powinni odbierać prawo wykonywania zawodu.

In [20]:
try:
    raise BaseException()
except:
    print("Caught")

Caught


In [21]:
try:
    a = int(input("Podaj a "))
    prunt("a do kwadratu wynosi", a**2)
except:
    print("Podałeś niepoprawne a")  # Czy aby na pewno? Miłego debuggowania.

Podałeś niepoprawne a


#### Blok except musi obsłużyć złapany wyjątek

Jeżeli wyjątek został złapany i obsłużony przez blok `except`, program wznawia normalne działanie od miejsca za tym blokiem. W związku z tym kod w `except` powinien doprowadzić program do stanu poprawnego, w którym można bezpiecznie działać dalej. Jeżeli nie jesteśmy w stanie tego zrobić, albo nie powinniśmy w ogóle łapać tego wyjątku, albo rzucić go dalej przy pomocy `raise`.

Błędem jest złapanie wyjątku nisko (blisko miejsca, w którym został rzucony), wypisanie komunikatu i kontynuowanie pracy, mimo że błąd tak naprawdę nie został naprawiony. (patrz także następna zasada)

In [22]:
import math

def perimeter(r):
    try:
        assert r > 0, "Promień musi być dodatni"
    except AssertionError:
        print("Promień musi być dodatni")
    return 2 * math.pi * r


print(perimeter(-1))

Promień musi być dodatni
-6.283185307179586


In [23]:
try:
    5 + '5'
except TypeError:
    print("I don't know what to do!")
    raise

I don't know what to do!


TypeError: unsupported operand type(s) for +: 'int' and 'str'

#### "Rzucaj wcześnie, łap późno"

Wyjątki należy rzucać najwcześniej jak to możliwe, czyli od razu, gdy jesteśmy w stanie wykryć błąd. Pozwala to najlepiej wyśledzić przyczynę błędu. Poza tym (prozaiczna kwestia) nie marnujemy czasu na obliczenia, które za chwilę i tak trzeba będzie przerwać, z powodu błędu, który dało się wykryć wcześniej.

Przykładowo:
- jeśli sprawdzamy poprawność argumentów funkcji, to róbmy to na samym jej początku, a nie dopiero, gdy potrzebujemy ich użyć
- jeżeli przyjmujemy wejście od użytkownika, to od razu skontrolujmy jego poprawność (użytkownikowi nie wolno ufać)

Wyjątki łapiemy wtedy, gdy jesteśmy w stanie jakoś naprawić błąd. Czasami jest to dość blisko miejsca ich rzucenia (np. jeśli błąd wystąpił podczas komunikacji z użytkownikiem), a czasami dopiero na najwyższym poziomie wykonania programu (jeżeli jedyne co możemy zrobić, to zakończyć program z jakimś zrozumiałym dla użytkownika komunikatem; to jest jedna z niewielu sytuacji, kiedy dopuszczalne jest łapanie wszystkich typów jednym blokiem `except`). Jeżeli pisząc funkcję mamy wątpliwość w jaki sposób wyjątek obsłużyć, to prawdopodobnie nie powinniśmy tego robić, tylko zostawić to kodowi, który będzie tę funkcję wywoływał.

#### Wyjątki nie są niczym złym

To nie wyjątki są złe, tylko awarie. Jeśli doszło do jakiejś sytuacji awaryjnej, to prawidłowe zasygnalizowanie jej, z przekazaniem odpowiednich informacji, pomoże wykryć jej źródło i naprawić błąd. Jest to dużo lepsze niż udawanie, że nic się nie stało i zwracanie złych wyników.

#### "Łatwiej prosić o wybaczenie, niż o pozwolenie"

W Pythonie przyjęło się, że nie sprawdzamy możliwości wykonania operacji, tylko próbujemy ją wykonać i obsługujemy ew. wyjątki. Przykładowo, przed odczytaniem elementu z listy nie sprawdzamy, czy indeks mieści się w odpowiednim zakresie, tylko jesteśmy gotowi na IndexError.

Konsekwencją tego podejścia jest fakt, że implementowane przez nas operacje nie powinny nigdy zwracać błędnych wyników, tylko rzucać wyjątki.

Wracając do przykładu z liczeniem obwodu koła:
- mnożenie liczby ujemnej przez pi nie powoduje rzucenia wyjątku, więc nie możemy się powołać na powyższą zasadę
- osoba wywołująca naszą funkcję ma prawo nie sprawdzać poprawności promienia, zakładając, że błędną wartość funkcja zasygnalizuje wyjątkiem

Lektura dodatkowa:

- https://www.pythonforthelab.com/blog/learning-not-to-handle-exceptions/
- https://realpython.com/python-exceptions/
- https://realpython.com/the-most-diabolical-python-antipattern/
- https://softwareengineering.stackexchange.com/questions/231057/exceptions-why-throw-early-why-catch-late
- https://pl.python.org/docs/tut/node10.html (Python 2)